## Objective

* Download OOI Array CTD Data 
* Transform related variables for machine learning QC.

## Method

* Take [Global Station Papa](http://ooi.visualocean.net/regions/view/GP) data as a testbed
* Download major CTD variables (e.g., temperature, salinity, depth) and save them as a csv file
* Output QC flags (normal or suspicious)
* [OOI data QC methods](https://oceanobservatories.org/quality-control/)



Here, I download CTD data from [OOI Station PAPA](https://ooinet.oceanobservatories.org/data_access/?search=Global%20Station%20Papa).

In [6]:
import warnings
warnings.filterwarnings('ignore')
import requests
import time
import numpy as np
from thredds_crawler.crawl import Crawl
import os
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from download_OOI_Array_CTD_data import request_data, download_data, suspicious_rate

# OOI data asscess setting
# OOI data team account 
username = 'OOIAPI-D8S960UXPK4K03'
token = 'IXL48EQ2XY'
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv'

In [9]:
# Find device address. Follow: https://github.com/ooi-data-review/m2m_demo/blob/master/notebooks/netcdf_data_request.ipynb
# You need to change this if you're looking for other instruments.
# See http://ooi.visualocean.net/instruments/view/GP02HYPM-WFP02-04-CTDPFL000

#Apex Profiler Mooring Wire-Following Profiler CTD

station_dict = {'array_name': ['Global-Station-Papa']*2,
               'ref_designator': ['GP02HYPM-WFP02-04-CTDPFL000', 
                                  'GP02HYPM-WFP03-04-CTDPFL000'],
               'method': ['telemetered']*2,
               'stream': ['ctdpf_ckl_wfp_instrument', 
                          'ctdpf_ckl_wfp_instrument'],
               'beginDT': ['2013-07-26T11:00:03.000Z', 
                          '2013-07-26T15:00:03.000Z'],
               'endDT': ['2016-01-04T19:08:08.955Z', 
                         '2016-08-23T13:50:47.789Z']}
station_df = pd.DataFrame(station_dict)
station_df.head()

,array_name,ref_designator,method,stream,beginDT,endDT
0,Global-Station-Papa,GP02HYPM-WFP02-04-CTDPFL000,telemetered,ctdpf_ckl_wfp_instrument,2013-07-26T11:00:03.000Z,2016-01-04T19:08:08.955Z
1,Global-Station-Papa,GP02HYPM-WFP03-04-CTDPFL000,telemetered,ctdpf_ckl_wfp_instrument,2013-07-26T15:00:03.000Z,2016-08-23T13:50:47.789Z


In [10]:
for index, x in station_df.iterrows():
    array_name = x['array_name']
    refdes = x['ref_designator']
    method = x['method']
    stream = x['stream']
    beginDT = x['beginDT']
    endDT = x['endDT']
    
    # Request data first
    data = request_data(username, token, base_url, refdes, method, stream, beginDT, endDT)
    # Download data from Thresdds server and save them as csv files
    df = download_data(data, array_name, refdes, method, stream)
    # Calcuate suspicious rate
    sp_df = suspicious_rate(df)
    print(sp_df)

Data request url is: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/GP02HYPM/WFP02/04-CTDPFL000/telemetered/ctdpf_ckl_wfp_instrument
request completed!
THREDDS server address: https://opendap.oceanobservatories.org/thredds/catalog/ooi/ooidatateam@gmail.com/20180823T204609-GP02HYPM-WFP02-04-CTDPFL000-telemetered-ctdpf_ckl_wfp_instrument/catalog.html
Data saved as Global-Station-Papa_GP02HYPM-WFP02-04-CTDPFL000_telemetered_ctdpf_ckl_wfp_instrument.csv
              suspicious_number  suspicious_rate
temperature                  22         0.000761
conductivity              14219         0.491684
pressure                     22         0.000761
salinity                     22         0.000761
density                      22         0.000761
Data request url is: https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/GP02HYPM/WFP03/04-CTDPFL000/telemetered/ctdpf_ckl_wfp_instrument
request completed!
THREDDS server address: https://opendap.oceanobservatories.org/thred

QC table
```
Test name              Bit position
                         15 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0
global_range_test         0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
dataqc_localrangetest     0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
dataqc_spiketest          0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
dataqc_polytrendtest      0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
dataqc_stuckvaluetest     0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
dataqc_gradienttest       0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
dataqc_propagateflags     0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
```

